## Load Documents - Split and Create Retriever on Vector Store

In [1]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

llama_loader = PyPDFLoader("llama2.pdf").load()  # Load the PDF document
llama_loader

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
pdf_documents = text_splitter.split_documents(llama_loader)  # Split the document into chunks
pdf_documents

openai_embeddings = OpenAIEmbeddings(model="text-embedding-3-large")  # Initialize embeddings

# Create a new vector store for the PDF documents
vector_store_pdf = Chroma(
    collection_name="llama2_collection",  # Name of the collection
    embedding_function=openai_embeddings,  # Embedding function
    persist_directory="./chroma_pdf_db"    # Directory to persist the database
)

vector_store_pdf.add_documents(pdf_documents)  # Add the PDF documents to the vector store





/var/folders/6p/ddx1yx2j547512yh8wlgztw00000gn/T/ipykernel_3005/3304255382.py:16: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store_pdf = Chroma(


['2f4788b0-ce83-496f-8c9a-e9fc9389bb95',
 '5c060cc9-592c-4238-b8d7-628aecd7a0a1',
 '6d2eb05d-6abe-43f8-96b4-fe0a8252df06',
 'ceee5484-ba41-470b-bfdd-f8ee8d405d9c',
 'f5f3d9df-6af8-4543-9a84-66072244ba9e',
 '2b22a3e5-20b9-4cc1-906d-ea6dcedcb292',
 'ef9231b5-2404-45b6-abdf-9d3c41467297',
 '0acfd4a2-9d7a-4586-aedf-76d245506b92',
 '1729850c-f1ad-495a-aa5b-7168eda0c084',
 'ec70487a-5345-415c-b63c-0f20f4968b3a',
 'ff0712dd-0741-422d-9c61-a9c1d027d56c',
 'e0882be1-8a42-42b0-a127-94ddcfcbdaed',
 'e5800773-7708-4618-8606-c6a20c97ffaa',
 'e88954a6-c4c1-4990-9bf4-0e3d43b8bd5d',
 'cab4b2d8-63b3-4193-a3f8-43794c49c8c9',
 '85b9b54e-5185-4e72-82d2-6b042902e003',
 'f1e85ccf-808a-40af-a943-41c0cd5eedbe',
 '7e56f4b4-800b-4972-a331-72ae67c752d0',
 'cab66e3a-6c46-44ef-8dd8-13af960b8bfd',
 'cd771ca5-3fb5-48f0-8097-8a2e12c56314',
 '5643561a-6632-435c-a256-ce2cdba257ff',
 '76cf80b2-f952-418c-a624-b6e4c6449fec',
 '0fdbd785-85cb-487c-8552-9de7fb352619',
 '82d4de03-30b5-412e-80f2-81bb75cf8efa',
 '648cad78-e9a9-

In [2]:
rag_retriever = vector_store_pdf.as_retriever(search_kwargs={"k": 5})  # Create a retriever

## Test Retriever

In [3]:
results = rag_retriever.invoke("what is llama model?")  # Invoke the retriever with a query
results

[Document(metadata={'total_pages': 77, 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'moddate': '2023-07-20T00:30:36+00:00', 'title': '', 'trapped': '/False', 'author': '', 'source': 'llama2.pdf', 'subject': '', 'keywords': '', 'creationdate': '2023-07-20T00:30:36+00:00', 'creator': 'LaTeX with hyperref', 'producer': 'pdfTeX-1.40.25', 'page': 3, 'page_label': '4'}, page_content='1. Llama 2, an updated version ofLlama 1, trained on a new mix of publicly available data. We also\nincreased the size of the pretraining corpus by 40%, doubled the context length of the model, and\nadopted grouped-query attention (Ainslie et al., 2023). We are releasing variants ofLlama 2with\n7B, 13B, and 70B parameters. We have also trained 34B variants, which we report on in this paper\nbut are not releasing.§\n2. Llama 2-Chat, a fine-tuned version ofLlama 2that is optimized for dialogue use cases. We release\nvariants of this model with 7B, 13B

## Langchain Prompt & Define model

In [4]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
# import pprint

prompt = hub.pull("rlm/rag-prompt")
# pprint.pprint(prompt.messages) 
llm = ChatOpenAI(model="gpt-4o", temperature=0)  # Initialize the language model


In [5]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
    

## Creating chain and invoke

In [6]:
# chain = rag_retriever | prompt | llm | StrOutputParser  # Create a chain with the prompt, LLM, and retriever


chain = ({"context": rag_retriever | format_docs , "question": RunnablePassthrough()}  # Define the chain structure with context and question
| prompt 
| llm 
| StrOutputParser())  # Create a chain with the prompt, LLM, and retriever




In [7]:
chain.invoke("what is llama model?")  # Invoke the chain with a query

'The Llama model, specifically Llama 2, is an updated version of Llama 1, trained on a new mix of publicly available data with an increased pretraining corpus size and context length. It includes variants with 7B, 13B, and 70B parameters, and a fine-tuned version called Llama 2-Chat optimized for dialogue use cases. Llama 2 is a large language model (LLM) that carries potential risks and benefits, and its responsible use is emphasized.'

In [8]:
chain.invoke("Write a fibnocci series in python")  # Invoke the chain with a query

'To write a Fibonacci series in Python, you can use the following code:\n\n```python\ndef fibonacci_series(n):\n    """Generate a Fibonacci series up to the nth number."""\n    fib_series = [0, 1]\n    while len(fib_series) < n:\n        fib_series.append(fib_series[-1] + fib_series[-2])\n    return fib_series[:n]\n\n# Example usage\nprint(fibonacci_series(10))\n```\n\nThis function generates a Fibonacci series up to the nth number and returns it as a list.'